In [100]:
import pandas as pd
import json
import re
from collections import defaultdict

In [101]:
d = defaultdict(lambda: defaultdict())

In [102]:
for state in ['WA','NSW','VIC','TAS','NT','ACT','SA', 'QLD']:

    ta = pd.read_csv(f'/Users/ik/Data/2016_GCP_POA_for_{state}_short-header/2016 Census GCP Postal Areas for {state}/2016Census_G04A_{state}_POA.csv')
    tb = pd.read_csv(f'/Users/ik/Data/2016_GCP_POA_for_{state}_short-header/2016 Census GCP Postal Areas for {state}/2016Census_G04B_{state}_POA.csv')

    t = ta.join(tb.set_index('POA_CODE_2016'), on='POA_CODE_2016', how='inner')

    for gender in 'M F'.split():
    
        males = t.loc[:, ['POA_CODE_2016'] + [_ for _ in t.columns 
                                              if (gender in _) and (_.count('_') > 3)][3:]]
        males['85+'] = males.iloc[:,-4:].sum(axis=1)
        males = males.drop(males.columns[-4:-1], axis=1)
    
        males = males.rename(lambda x: '_'.join([v for v in x.split('_') if v.isdigit()]) 
                         if ('+' not in x) and ('2016' not in x) 
                             else x, axis='columns').rename(columns={'POA_CODE_2016': 'postcode'})
    
        males['postcode'] = males['postcode'].apply(lambda x: ''.join([v for v in x if v.isdigit()]))
    
        males.head()
    
        d[gender.lower()].update(males.set_index('postcode').to_dict(orient='index'))

In [103]:
json.dump(d, open('abs_population_by_postcode.json','w'))